# Content
* Importing Libraries
* Constants
* Preprocessing
    * Normalizing
    * Tokenizing
    * Stemming
    * Lemmatizing
* Feature Engineering
    * Bag of Words
    * FastText Word2Vec
* Model Selection

# Importing Libraries

In [2]:
from __future__ import unicode_literals

import json
import os
import numpy as np
import re
import pandas as pd
from functools import reduce
from hazm import *
from pprint import pprint

# Constants

In [11]:
# Data root path
data_root = 'data'

# Dataset dataframe column names
keys = None

# News headline tags
valid_tags = None

# News agencies
news_agencies = None

# Preprocessing

### Import Dataset

In [12]:
with open(os.path.join(data_root, 'out.jsonl'), encoding='utf-8') as json_data:
    news = [json.loads(line) for line in json_data]
    news = pd.DataFrame(news)
print('Number of Datapoints: {}'.format(len(news)))

Number of Datapoints: 1000


In [13]:
keys = list(news.columns)
pd.DataFrame([keys])

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,NewsAgency,_id,body,bodyHtml,date,newsCode,newsLink,newsPath,newsPathLinks,rutitr,subtitle,tags,title


Lets look at our data

In [14]:
news.head(2)

,NewsAgency,_id,body,bodyHtml,date,newsCode,newsLink,newsPath,newsPathLinks,rutitr,subtitle,tags,title
0,AsrIran,5b4f7279020eb20597f401b4,مدیرعامل سابق استقلال از عضویت در هیات مدیره ا...,"<img align=""left"" class=""news_corner_image"" s...",تاریخ انتشار: ۲۱:۲۲ - ۲۷ تير ۱۳۹۷ - 18 July 2018,621662,http://www.asriran.com/fa/news/621662,صفحه نخست » ورزشی,"{'صفحه نخست': '/fa/archive?service_id=1', 'ورز...",,,"{'استقلال': '/fa/tag/1/استقلال', 'افتخاری': '/...",افتخاری قید ماندن در هیات مدیره استقلال را هم زد
1,AsrIran,5b4f7279020eb20597f401b5,دادستان انتظامی مالیاتی سازمان امور مالیاتی گف...,<p><br/>دادستان انتظامی مالیاتی سازمان امور م...,تاریخ انتشار: ۲۱:۱۱ - ۲۷ تير ۱۳۹۷ - 18 July 2018,621659,http://www.asriran.com/fa/news/621659,صفحه نخست » اجتماعی,"{'صفحه نخست': '/fa/archive?service_id=1', 'اجت...",,,"{'مالیات': '/fa/tag/1/مالیات', 'دادستان': '/fa...",دادستان انتظامی مالیاتی سازمان مالیات: آخرین ا...


### subtitle & rutitr

In [15]:
print(news.subtitle.sample(5))
print(news.rutitr.sample())

145    علیرضا جهانبخش ستاره فوتبال ایران و یکی از موف...
818    البته در حال حاضر برنامه فتوشاپ اکسپرس برروی ت...
245                                                     
375                                                     
971                                                     
Name: subtitle, dtype: object
706    
Name: rutitr, dtype: object


As we can see, there might not exist any 'subtitle' or 'rutitr', so we drop them if they do not have valuable features.

In [16]:
print("Not null 'subtitle' ",len([i for i in news.subtitle if len(i) != 0]))
print("Not null 'rutitr' ",len([i for i in news.rutitr if len(i) != 0]))

Not null 'subtitle'  569
Not null 'rutitr'  50


So based on information we got here, we know that these columns can help us, so we consider them.

### Drop Useless Columns

But it is clear for us, `date`,`newsCode`, `newsLink`,`bodyHtml` and `_id` are useless features. So we remove them from our dataset.


In [17]:
news = news.drop(['_id','date','newsCode','newsLink','bodyHtml'], axis=1)

In [18]:
news.head(5)

,NewsAgency,body,newsPath,newsPathLinks,rutitr,subtitle,tags,title
0,AsrIran,مدیرعامل سابق استقلال از عضویت در هیات مدیره ا...,صفحه نخست » ورزشی,"{'صفحه نخست': '/fa/archive?service_id=1', 'ورز...",,,"{'استقلال': '/fa/tag/1/استقلال', 'افتخاری': '/...",افتخاری قید ماندن در هیات مدیره استقلال را هم زد
1,AsrIran,دادستان انتظامی مالیاتی سازمان امور مالیاتی گف...,صفحه نخست » اجتماعی,"{'صفحه نخست': '/fa/archive?service_id=1', 'اجت...",,,"{'مالیات': '/fa/tag/1/مالیات', 'دادستان': '/fa...",دادستان انتظامی مالیاتی سازمان مالیات: آخرین ا...
2,AsrIran,قیمت سبد نفتی اوپک دیروز به روند کاهشی خود ادا...,صفحه نخست » اقتصادی,"{'صفحه نخست': '/fa/archive?service_id=1', 'اقت...",,,"{'اوپک': '/fa/tag/1/اوپک', 'نفت': '/fa/tag/1/ن...",قیمت سبد نفتی اوپک یک گام دیگر عقب نشست/ 70 دل...
3,AsrIran,رئیس فراکسیون فرهنگیان مجلس خطاب به وزیر آموزش...,صفحه نخست » اجتماعی,"{'صفحه نخست': '/fa/archive?service_id=1', 'اجت...",,,"{'آموزش و پرورش': '/fa/tag/1/آموزش و پرورش', '...",حاجی‌بابایی به بطحایی:آقای وزیر در اطلاع‌رسانی...
4,AsrIran,رئیس صندوق بین&zwnj;المللی پول در آستانه نشست ...,صفحه نخست » بین الملل,"{'صفحه نخست': '/fa/archive?service_id=1', 'بین...",,,{'صندوق بین المللی پول': '/fa/tag/1/صندوق بین ...,رئیس صندوق بین‌المللی پول: آمریکا از جنگ تعرفه...


### newsPath & newsPathLinks

In [19]:
newspathlinks_tags = list(set([list(x.keys())[1] for x in news.newsPathLinks]))
newspath_tags = list(set(x.split(' » ')[1] for x in news.newsPath))
print("news path links and news path show the same thing? => ", newspath_tags == newspathlinks_tags)
valid_tags = newspath_tags
pd.DataFrame([valid_tags])

news path links and news path show the same thing? =>  True


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,خواندنی ها و دیدنی ها,ورزشی,روانشناسی,علمی,فناوری و IT,حوادث,سلامت,داستان کوتاه,اقتصادی,فرهنگی/هنری,عمومی,دانلود,سیاسی,سیاست خارجی,بین الملل,اجتماعی,سرگرمی


Note: we can remove `newsPathLinks` because it is exactly duplicate of `newsPath` and update `newsPath` column in dataframe with just one keyword

In [20]:
news = news.drop(['newsPathLinks'], axis=1)
news.head(2)

,NewsAgency,body,newsPath,rutitr,subtitle,tags,title
0,AsrIran,مدیرعامل سابق استقلال از عضویت در هیات مدیره ا...,صفحه نخست » ورزشی,,,"{'استقلال': '/fa/tag/1/استقلال', 'افتخاری': '/...",افتخاری قید ماندن در هیات مدیره استقلال را هم زد
1,AsrIran,دادستان انتظامی مالیاتی سازمان امور مالیاتی گف...,صفحه نخست » اجتماعی,,,"{'مالیات': '/fa/tag/1/مالیات', 'دادستان': '/fa...",دادستان انتظامی مالیاتی سازمان مالیات: آخرین ا...


In [21]:
news.loc[:,'newsPath'] = list(x.split(' » ')[1] for x in news.newsPath)
news.head(3)

,NewsAgency,body,newsPath,rutitr,subtitle,tags,title
0,AsrIran,مدیرعامل سابق استقلال از عضویت در هیات مدیره ا...,ورزشی,,,"{'استقلال': '/fa/tag/1/استقلال', 'افتخاری': '/...",افتخاری قید ماندن در هیات مدیره استقلال را هم زد
1,AsrIran,دادستان انتظامی مالیاتی سازمان امور مالیاتی گف...,اجتماعی,,,"{'مالیات': '/fa/tag/1/مالیات', 'دادستان': '/fa...",دادستان انتظامی مالیاتی سازمان مالیات: آخرین ا...
2,AsrIran,قیمت سبد نفتی اوپک دیروز به روند کاهشی خود ادا...,اقتصادی,,,"{'اوپک': '/fa/tag/1/اوپک', 'نفت': '/fa/tag/1/ن...",قیمت سبد نفتی اوپک یک گام دیگر عقب نشست/ 70 دل...


### NewsAgency

In [22]:
news_agencies = list(news.NewsAgency.unique())
pd.DataFrame([news_agencies])

,0
0,AsrIran


### tags

In [23]:
def tag_extractor(tags_dict):
    """
    gets a tags dictionary and finds unique tags in collection of values and keys
    
    ::params tags_dict : 
    """
    keys = list(set(tags_dict.keys()))
    values = { v.split('/')[-1] for v in set(tags_dict.values())}
    [values.add(i) for i in keys]
    return list(values)

print('not processed tags_dict',news.tags[0])
print('processesd tags_dict',tag_extractor(news.tags[0]))

not processed tags_dict {'استقلال': '/fa/tag/1/استقلال', 'افتخاری': '/fa/tag/1/افتخاری'}
processesd tags_dict ['افتخاری', 'استقلال']


Now we replace `tags` column with extract values from `tag_extractor` function.

In [24]:
news.loc[:, 'tags'] = [tag_extractor(tag) for tag in news.tags]

In [27]:
news.head(5)

,NewsAgency,body,newsPath,rutitr,subtitle,tags,title
0,AsrIran,مدیرعامل سابق استقلال از عضویت در هیات مدیره ا...,ورزشی,,,"[افتخاری, استقلال]",افتخاری قید ماندن در هیات مدیره استقلال را هم زد
1,AsrIran,دادستان انتظامی مالیاتی سازمان امور مالیاتی گف...,اجتماعی,,,"[دادستان, مالیات]",دادستان انتظامی مالیاتی سازمان مالیات: آخرین ا...
2,AsrIran,قیمت سبد نفتی اوپک دیروز به روند کاهشی خود ادا...,اقتصادی,,,"[نفت, اوپک]",قیمت سبد نفتی اوپک یک گام دیگر عقب نشست/ 70 دل...
3,AsrIran,رئیس فراکسیون فرهنگیان مجلس خطاب به وزیر آموزش...,اجتماعی,,,"[فرهنگیان, آموزش و پرورش]",حاجی‌بابایی به بطحایی:آقای وزیر در اطلاع‌رسانی...
4,AsrIran,رئیس صندوق بین&zwnj;المللی پول در آستانه نشست ...,بین الملل,,,"[امریکا, صندوق بین المللی پول]",رئیس صندوق بین‌المللی پول: آمریکا از جنگ تعرفه...


<font color='yellow'>Note:</font> If you look at row 0 and row 10, you can see there is many noise in this dataset. We have two different tags for same news.

## Normalizing

In [28]:
normalizer = Normalizer()
news['body'] = news['body'].apply(normalizer.normalize)
news['rutitr'] = news['rutitr'].apply(normalizer.normalize)
news['subtitle'] = news['subtitle'].apply(normalizer.normalize)
news['title'] = news['title'].apply(normalizer.normalize)

## Tokenizing

In [29]:
def tokenize(phrase):
    sentences = sent_tokenize(phrase)
    if len(sentences) > 1:
        words = reduce(np.append, [word_tokenize(sentence) for sentence in sentences])
    elif len(sentences) == 1:
        words = word_tokenize(sentences[0])
    else:
        words = None
    return words

In [30]:
news['body'] = news['body'].apply(tokenize)
news['rutitr'] = news['rutitr'].apply(tokenize)
news['subtitle'] = news['subtitle'].apply(tokenize)
news['title'] = news['title'].apply(tokenize)

## Stemming

In [31]:
stemmer = Stemmer()
stem = lambda s: [stemmer.stem(w) for w in s] if s is not None else None
news['body'] = news['body'].apply(stem)
news['rutitr'] = news['rutitr'].apply(stem)
news['subtitle'] = news['subtitle'].apply(stem)
news['title'] = news['title'].apply(stem)

## Lemmatizing

In [32]:
lemmatizer = Lemmatizer()
lemmatize = lambda s: [lemmatizer.lemmatize(w) for w in s] if s is not None else None
news['body'] = news['body'].apply(lemmatize)
news['rutitr'] = news['rutitr'].apply(lemmatize)
news['subtitle'] = news['subtitle'].apply(lemmatize)
news['title'] = news['title'].apply(lemmatize)

In [36]:
news.head(5)

,NewsAgency,body,newsPath,rutitr,subtitle,tags,title
0,AsrIran,"[مدیرعامل, سابق, استقلال, از, عضو, در, ه, مدیر...",ورزشی,None,None,"[افتخاری, استقلال]","[افتخار, قید, ماندن, در, ه, مدیره, استقلال, را..."
1,AsrIran,"[دادس, انتظام, مالیات, ساز, امور, مالیات, گف, ...",اجتماعی,None,None,"[دادستان, مالیات]","[دادس, انتظام, مالیات, ساز, مال, :, آخرین, اخط..."
2,AsrIran,"[قیم, سبد, نفت, اوپک, دیروز, به, روند, کاهش, خ...",اقتصادی,None,None,"[نفت, اوپک]","[قیم, سبد, نفت, اوپک, یک, گا, دیگر, عقب, نشست/..."
3,AsrIran,"[رئیس, فراکسیون, فرهنگ, مجلس, خطاب, به, وزیر, ...",اجتماعی,None,None,"[فرهنگیان, آموزش و پرورش]","[حاجی‌بابا, به, بطحا, :, آقا, وزیر, در, اطلاع‌..."
4,AsrIran,"[رئیس, صندوق, بین&zwnj;الملل, پول, در, آستانه,...",بین الملل,None,None,"[امریکا, صندوق بین المللی پول]","[رئیس, صندوق, بین‌الملل, پول, :, آمریکا, از, ج..."


## Removing Stopwords
For this step, we use stopwords from <a href='https://github.com/kharazi/persian-stopwords'>this repository</a>.

There are some files of stopwords and we are using <a href='https://github.com/kharazi/persian-stopwords/blob/master/persian'>this</a>.

```bash
git clone https://github.com/kharazi/persian-stopwords.git
```

In [96]:
stopwords_root = 'persian-stopwords'

with open(os.path.join(stopwords_root, 'persian'), encoding='utf-8') as stopwords_file:
    stopwords = [re.sub(r'\n','',word) for word in stopwords_file]
pd.DataFrame([stopwords[150:170]])

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,اید,ایشان,ایم,این,این جوری,این قدر,این گونه,اینان,اینجا,اینجاست,ایند,اینطور,اینقدر,اینها,اینهاست,اینو,اینچنین,اینک,اینکه,اینگونه


Now we remove all stopwords from our dataset.

In [101]:
def filter_words(words_list, stopwords=stopwords):
    """
    Gets a list of words and remove stopwords from that list.
    
    :param words_list: a list of words to apply stopwords
    :param stopwords: a list of stopwords to remove from words_list
    """
    if words_list is None:
        return None
    filtered_words = [word for word in words_list if word not in stopwords]
    return filtered_words

s = news.title[0]
print(s)
s_filtered = filter_words(s, stopwords)
print(s_filtered)

['افتخار', 'قید', 'ماندن', 'در', 'ه', 'مدیره', 'استقلال', 'را', 'ه', 'زد#زن']
['افتخار', 'قید', 'ماندن', 'مدیره', 'استقلال', 'زد#زن']


In [102]:
news['body'] = news['body'].apply(filter_words)
news['rutitr'] = news['rutitr'].apply(filter_words)
news['subtitle'] = news['subtitle'].apply(filter_words)
news['title'] = news['title'].apply(filter_words)

In [103]:
news.title[0]

['افتخار', 'قید', 'ماندن', 'مدیره', 'استقلال', 'زد#زن']

# Feature Engineering
The inherent unstructured (no neatly formatted data columns!) and noisy nature of textual data makes it harder for machine learning methods to directly work on raw text data.

##### Motivation
The importance of feature engineering is even more important for unstructured, textual data because we need to convert free flowing text into some numeric representations which can then be understood by machine learning algorithms.

### Feature Engineering Strategies
We try different methods and compare result using F1, precision and recall score gathered by result of different machine learning methods.

#### Import Libraries

In [38]:
import pandas as pd
import numpy as np
import re
import nltk
import matplotlib.pyplot as plt

%matplotlib inline

['arabic',
 'azerbaijani',
 'danish',
 'dutch',
 'english',
 'finnish',
 'french',
 'german',
 'greek',
 'hungarian',
 'indonesian',
 'italian',
 'kazakh',
 'nepali',
 'norwegian',
 'portuguese',
 'romanian',
 'russian',
 'spanish',
 'swedish',
 'turkish']

# Model Selection

## Naive Bayes

## Logistic Regression

## SVM

## Simple ANN

## CNN

## RNN

## CNN + RNN